<a href="https://colab.research.google.com/github/AbhiJ2706/snake_game_reinforcement_learning/blob/main/SnakeRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import randint
from random import choice
from random import uniform
from random import shuffle
from itertools import combinations
from copy import deepcopy
import math

In [ ]:
n = 7

grid = [[-1 for i in range(n)]] + [[-1] + [0 for i in range(n - 2)] + [-1] for i in range(n - 2)] + [[-1 for i in range(n)]]

print(grid)

all_apples = [(x, y) for x in range(1, n - 1) for y in range(1, n - 1)]

apple_no = 0
shuffle(all_apples)
apple_pos = all_apples[0]
radius = ([*range(1, apple_pos[0] - 1), *range(apple_pos[0] + 1, n - 1)], 
          [*range(1, apple_pos[1] - 1), *range(apple_pos[1] + 1, n - 1)])
snake_pos = (choice(radius[0]), choice(radius[1]))

grid[apple_pos[0]][apple_pos[1]] = 1
grid[snake_pos[0]][snake_pos[1]] = -1

original_snake_pos = deepcopy(snake_pos)
original_apple_pos = deepcopy(apple_pos)

snake = [snake_pos]

# CONVENTION: L U R D

dirs = ((-1, 0), (0, -1), (1, 0), (0, 1))

print(snake_pos, apple_pos)

[[-1, -1, -1, -1, -1, -1, -1], [-1, 0, 0, 0, 0, 0, -1], [-1, 0, 0, 0, 0, 0, -1], [-1, 0, 0, 0, 0, 0, -1], [-1, 0, 0, 0, 0, 0, -1], [-1, 0, 0, 0, 0, 0, -1], [-1, -1, -1, -1, -1, -1, -1]]
(1, 4) (4, 1)


In [ ]:
snake = [snake_pos]

def move_snake(snake, dir):
    return list(filter(lambda x: x is not None, [(snake[0][0] + dir[0], snake[0][1] + dir[1])] + \
            list(map(lambda x: snake[x[0]] if x[0] < len(snake) - 1 else None, \
                     enumerate(snake, 0)))))

# CONVENTION: L U R D

dirs = ((0, -1), (-1, 0), (0, 1), (1, 0))

epsilon = 0.2

lr = 0.13

discount = 0.6


states = list(set(map(lambda x: tuple(map(lambda y: 0 if y % 2 == 0 else -1, x)), combinations(range(8), 4))))

apple_states = []

for s in states:
    apple_states += [tuple(1 if j == i else s[i] for i in range(len(s))) for j in range(len(s))]

states += apple_states

print(all_apples)

for i in range(2):
    all_states = list(map(lambda x: [(*x, *a) for a in all_apples], states))

    states = []

    for s in all_states:
        states += s

states = {s: [0 for i in range(4)] for s in states}

print("length of states: ", len(states))

def reward(s, a):
    global snake, apple_pos
    head = snake[0]
    if s[a] == 1:
        return 100
    elif s[a] == 0:
        new_head = (head[0] + dirs[a][0], head[1] + dirs[a][1])
        old_apple_dist = ((apple_pos[0] - head[0]) ** 2, (apple_pos[1] - head[1]) ** 2)
        new_apple_dist = ((apple_pos[0] - new_head[0]) ** 2, (apple_pos[1] - new_head[1]) ** 2)
        if old_apple_dist[0] < new_apple_dist[0] or old_apple_dist[1] < new_apple_dist[1]:
            return -10
        else:
            return 1
    else:
        return -100

def Q(s, a):
    global snake, grid, states, lr, discount, apple_pos
    if type(a) is range:
        return [Q(s, x) for x in a]
    else:
        head = (snake[0][0] + dirs[a][0], snake[0][1] + dirs[a][1])
        try:
            new_state = (grid[head[0]][head[1] - 1], grid[head[0] - 1][head[1]], grid[head[0]][head[1] + 1], grid[head[0] + 1][head[1]], *apple_pos, *head)
            #print(lr, reward(s, a), discount, max([states[new_state][i] for i in range(4)]), states[s][a])
            return lr * (reward(s, a) + discount * max(states[new_state]) - states[s][a])
        except:
            #print("here")
            return 0

def try_reference_spot(x, y):
    global grid
    try:
        z = grid[x][y]
        return True
    except:
        return False

def update_q_table(random=True):
    global snake, grid, states, apple_pos
    x = uniform(0, 1) if random else 0
    head = snake[0]
    state = (grid[head[0]][head[1] - 1], grid[head[0] - 1][head[1]], grid[head[0]][head[1] + 1], grid[head[0] + 1][head[1]], *apple_pos, *head)
    if x < epsilon and random:
        move = randint(0, 3)
        # make sure move doesn't force out of bounds condition
        while not try_reference_spot(snake[0][0] + dirs[move][0], snake[0][1] + dirs[move][1]):
            move = randint(0, 3)
        #print(Q(s, move))
        states[state][move] = (1 - lr) * states[state][move] + Q(state, move)
        return move
    else:
        move = max(enumerate(states[state], 0), key=lambda x: x[1])[0]
        if not random: 
            print(state, states[state], move)
        states[state][move] = (1 - lr) * states[state][move] + Q(state, move)
        return move

def iterate(print_out=False, random=True):
    global snake, grid, apple_pos, original_apple_pos, original_snake_pos, dirs
    alive = True

    while alive:
        move = update_q_table(random=random)
        #print(move)
        snake = move_snake(snake, dirs[move])
        if grid[snake[0][0]][snake[0][1]] == -1:
            shuffle(all_apples)
            apple_pos = all_apples[0]
            radius = ([*range(1, apple_pos[0] - 1), *range(apple_pos[0] + 1, n - 1)], 
                    [*range(1, apple_pos[1] - 1), *range(apple_pos[1] + 1, n - 1)])
            snake_pos = (choice(radius[0]), choice(radius[1]))
            snake = [snake_pos]
            alive = False
        elif grid[snake[0][0]][snake[0][1]] == 1:
            radius = ([*range(1, snake[0][0] - 1), *range(snake[0][0] + 1, n - 1)], 
                        [*range(1, snake[0][1] - 1), *range(snake[0][1] + 1, n - 1)])
            new_x = choice(radius[0])
            new_y = choice(radius[1])
            while grid[new_x][new_y] != 0:
                new_x = choice(radius[0])
                new_y = choice(radius[1])
            apple_pos = (new_x, new_y)
            tail = snake[-1]
            state = (grid[tail[0]][tail[1] - 1], grid[tail[0] - 1][tail[1]], grid[tail[0]][tail[1] + 1], grid[tail[0] + 1][tail[1]])
            for i in range(4):
                if state[i] == 0:
                    grid[tail[0] + dirs[i][0]][tail[1] + dirs[i][1]] = -1
                    new_state = (grid[snake[0][0]][snake[0][1] - 1], grid[snake[0][0] - 1][snake[0][1]], grid[snake[0][0]][snake[0][1] + 1], grid[snake[0][0] + 1][snake[0][1]])
                    if new_state == (-1, -1, -1, -1): continue
                    snake.append((tail[0] + dirs[i][0], tail[1] + dirs[i][1]))
                    break
        grid = [[-1 for i in range(n)]] + [[-1] + [0 for i in range(n - 2)] + [-1] for i in range(n - 2)] + [[-1 for i in range(n)]]
        grid[apple_pos[0]][apple_pos[1]] = 1
        if not random: print(snake)
        if alive:
            for s in snake:
                #print(s, move, apple_pos)
                grid[s[0]][s[1]] = -1
            if print_out: 
                for row in grid:
                    print(row)

    

[(4, 5), (3, 2), (3, 4), (5, 1), (3, 3), (2, 1), (2, 4), (1, 1), (2, 3), (4, 4), (5, 3), (1, 2), (1, 3), (4, 2), (4, 1), (5, 4), (4, 3), (2, 5), (5, 2), (1, 4), (3, 1), (2, 2), (3, 5), (1, 5), (5, 5)]
length of states:  30000


In [ ]:
for i in range(250000):
    print(i)
    iterate()

#print(states[(0, 0, 0, 0, 3, 5, 3, 5)])
print(len(states))

Streaming output truncated to the last 5000 lines.
245001
245002
245003
245004
245005
245006
245007
245008
245009
245010
245011
245012
245013
245014
245015
245016
245017
245018
245019
245020
245021
245022
245023
245024
245025
245026
245027
245028
245029
245030
245031
245032
245033
245034
245035
245036
245037
245038
245039
245040
245041
245042
245043
245044
245045
245046
245047
245048
245049
245050
245051
245052
245053
245054
245055
245056
245057
245058
245059
245060
245061
245062
245063
245064
245065
245066
245067
245068
245069
245070
245071
245072
245073
245074
245075
245076
245077
245078
245079
245080
245081
245082
245083
245084
245085
245086
245087
245088
245089
245090
245091
245092
245093
245094
245095
245096
245097
245098
245099
245100
245101
245102
245103
245104
245105
245106
245107
245108
245109
245110
245111
245112
245113
245114
245115
245116
245117
245118
245119
245120
245121
245122
245123
245124
245125
245126
245127
245128
245129
245130
245131
245132
245133
245134
245135
2451

In [ ]:
for i in range(2):
    iterate(print_out=True, random=False)
    print("-" * 100)

(0, 0, 0, 0, 4, 5, 2, 3) [-4.6748151475115005, -4.666265007092136, 2.0449999999999973, 2.0449999999999866] 2
[(2, 4)]
[-1, -1, -1, -1, -1, -1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, -1, 0, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, 0, 1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, -1, -1, -1, -1, -1, -1]
(0, 0, 0, 0, 4, 5, 2, 4) [-4.389539972065805, -4.38989394230961, 5.149999999999995, 5.149999999999995] 2
[(2, 5)]
[-1, -1, -1, -1, -1, -1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, 0, -1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, 0, 1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, -1, -1, -1, -1, -1, -1]
(0, 0, -1, 0, 4, 5, 2, 5) [-3.4550000042079008, -5.0, 0.0, 15.499999999999986] 3
[(3, 5)]
[-1, -1, -1, -1, -1, -1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, 0, 0, 0, 0, -1, -1]
[-1, 0, 0, 0, 0, 1, -1]
[-1, 0, 0, 0, 0, 0, -1]
[-1, -1, -1, -1, -1, -1, -1]
(0, 0, -1, 1, 4, 5, 3, 5) [-0.35477636046867383, -5.0, 0.0, 49.99999999999997] 3
[(4, 5), (4, 4)]
[-1, -1, -1, -1, -1, -1, -1]
[-1, 0, 1, 0, 